In [119]:
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

# from faker import Faker
# import random
# from tqdm import tqdm
# from babel.dates import format_date
# import matplotlib.pyplot as plt
# %matplotlib inline


In [2]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [3]:
words, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [158]:
print(np.shape(word_to_vec_map["father"]))

(50,)


In [80]:
input_sentences=[]

filepath = 'inp_dat.txt'
with open(filepath) as fp:
    line = fp.readline()
    cnt = 1
    while cnt <1000 :
        line = fp.readline()
        if cnt >15:
            input_sentences.append(line)
        cnt+=1
print(input_sentences[0])

Yes, I'm coming.



In [88]:
def wordcleaner(word):
    temp=""
    for char in word:
#         print(char)
        if char!= "," and char!= "!" and char!= "?" and char!= "." and char!= "'" and char!= "-":
#         if char!=',':
            temp+=char
    return temp
wordcleaner("i'm")

'im'

In [98]:
post_emb_list=[]

for l in input_sentences:
#     print(l)
    temp=[]
    for word in l.lower().split():
#         print(word)
        word = wordcleaner(word)
#         if word!="":
        w = word_to_vec_map.get(word)
        temp.append(w)
#         print(np.shape(temp))
    post_emb_list.append(temp)
    

In [174]:
max_len=20

In [199]:
batch_size = 500
context = np.zeros((500,max_len))
response = np.zeros((500,max_len))


In [200]:
# add = np.zeros((1,50))
# for l in post_emb_list:
#     while np.shape(l)!=(10,):
#         l.append(add)
    
# print(np.shape(post_emb_list[3]))

In [216]:
vocab_size = 10000
embedding_dim = 100
max_length = 20
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(input_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(input_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type,padding="post")
print(padded[6])
print(np.shape(padded))

index_word = {}
for word , index in word_index.items():
    index_word[index] = word

[ 17 287 145   5 288  38 146 450   7  39  71   0   0   0   0   0   0   0
   0   0]
(984, 20)


In [202]:
index=0
for idx in range(0,950,2):
    context[index,:] = padded[idx]
    response[index,:] = padded[idx+1]
    index +=1
    
print(np.shape(context))
print(context[6])
print(response[0])
response = list(response.swapaxes(0,1))


(500, 20)
[216. 291.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]
[  9. 285.  17. 447. 448.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]


In [190]:
import os
embeddings_index = {}
f = open( 'glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [191]:
embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [114]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            50,
                            weights=[embedding_matrix],
                            input_length=20,
                            trainable=False)

In [135]:
Tx = 20
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(20, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)


In [136]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator(inputs =([a,s_prev]))
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context


In [137]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [138]:
post_activation_LSTM_cell = LSTM(128, return_state = True) # post-attention LSTM 
output_layer = Dense(20000, activation=softmax)

In [139]:

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):

    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    e = embedding_layer(X)
    a = Bidirectional(LSTM(units = n_a ,return_sequences = True))(e)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs = context,initial_state = [s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X,s0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [140]:
model = model(20, 20,128, 128, 20000, 20000)


In [141]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 50)       48700       input_8[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 20, 256)      183296      embedding_1[3][0]                
____________________________________________________________________________________________

In [142]:
opt = Adam(lr = 0.005, beta_1 = 0.9,beta_2 = 0.999 , decay = 0.01)
model.compile(optimizer = opt , loss = "sparse_categorical_crossentropy" ,metrics = ['accuracy'])

In [204]:
s0 = np.zeros((500, 128))
c0 = np.zeros((500, 128))

In [206]:
model.fit([context,s0,c0],response,epochs=10,batch_size=50)

Epoch 1/10
500/500 [==============================] - 3s 7ms/step - loss: 36.1461 - dense_6_loss: 0.2844 - dense_6_accuracy: 0.0520 - dense_6_accuracy_1: 0.1740 - dense_6_accuracy_2: 0.4060 - dense_6_accuracy_3: 0.5660 - dense_6_accuracy_4: 0.6880 - dense_6_accuracy_5: 0.7780 - dense_6_accuracy_6: 0.8440 - dense_6_accuracy_7: 0.8920 - dense_6_accuracy_8: 0.9280 - dense_6_accuracy_9: 0.9560 - dense_6_accuracy_10: 0.9660 - dense_6_accuracy_11: 0.9800 - dense_6_accuracy_12: 0.9880 - dense_6_accuracy_13: 0.9900 - dense_6_accuracy_14: 0.9920 - dense_6_accuracy_15: 0.9960 - dense_6_accuracy_16: 1.0000 - dense_6_accuracy_17: 1.0000 - dense_6_accuracy_18: 1.0000 - dense_6_accuracy_19: 1.0000
Epoch 2/10
500/500 [==============================] - 3s 7ms/step - loss: 35.1077 - dense_6_loss: 0.0323 - dense_6_accuracy: 0.0520 - dense_6_accuracy_1: 0.1740 - dense_6_accuracy_2: 0.4060 - dense_6_accuracy_3: 0.5660 - dense_6_accuracy_4: 0.6880 - dense_6_accuracy_5: 0.7780 - dense_6_accuracy_6: 0.8440 -

In [207]:
import math
def beam_search_decoder(predictions, top_k = 3):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(20000 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [244]:
s = [("hello what is your name")]

In [245]:
seq = tokenizer.texts_to_sequences(s)
pad = pad_sequences(seq,maxlen=max_len, truncating="post",padding = 'post')

In [246]:
pad

array([[177,  11,  14,  18, 541,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [247]:
pred = model.predict([pad , np.zeros((1,128)) , np.zeros((1,128))])


In [248]:
pred

[array([[4.6369505e-01, 2.3353233e-07, 1.2653843e-02, ..., 2.4990908e-07,
         2.3885025e-07, 2.2141731e-07]], dtype=float32),
 array([[7.983412e-01, 8.883726e-09, 8.363408e-03, ..., 9.683281e-09,
         9.141195e-09, 8.302225e-09]], dtype=float32),
 array([[8.3223248e-01, 5.1626796e-09, 7.5285234e-03, ..., 5.6432397e-09,
         5.3172422e-09, 4.8140927e-09]], dtype=float32),
 array([[8.3650696e-01, 4.7878204e-09, 7.4157636e-03, ..., 5.2355142e-09,
         4.9317905e-09, 4.4631903e-09]], dtype=float32),
 array([[8.3707994e-01, 4.7390496e-09, 7.4005094e-03, ..., 5.1824700e-09,
         4.8816369e-09, 4.4175414e-09]], dtype=float32),
 array([[8.3715737e-01, 4.7324873e-09, 7.3984489e-03, ..., 5.1753131e-09,
         4.8748867e-09, 4.4113904e-09]], dtype=float32),
 array([[8.3716756e-01, 4.7315885e-09, 7.3981546e-03, ..., 5.1743494e-09,
         4.8739697e-09, 4.4105608e-09]], dtype=float32),
 array([[8.3716899e-01, 4.7314699e-09, 7.3981183e-03, ..., 5.1742206e-09,
         4.8738

In [249]:

ans = beam_search_decoder(pred)

In [250]:
ans

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -4.199701524377367),
 ([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -7.800967709777046),
 ([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -8.138833443227645)]

In [251]:
index_word[2]
# index_word[3]

'you'